# 泛化
:label:`sec_generalization_basics`

考虑两名大学生正在认真准备期末考试。通常，这种准备将包括通过做往年试题来练习和测试他们的能力。然而，做好往年的试题并不能保证他们在真正重要的时候也能表现出色。例如，想象一个学生，Ellie，她的准备完全由记忆往年试题的答案组成。即使Ellie拥有非凡的记忆力，能够完美回忆起任何*见过的*问题的答案，当面对一个新的（*以前没见过的*）问题时，她可能仍然会不知所措。相比之下，想象另一个学生，Irene，她的记忆力相对较差，但擅长发现规律。注意，如果考试真的只包含往年试题的重复题，那么Ellie的表现将轻松超过Irene。即使Irene推断出的模式产生了90%准确的预测，它们也无法与Ellie 100%的回忆相竞争。然而，即使考试完全由新题目组成，Irene也可能保持90%的平均分。

作为机器学习科学家，我们的目标是发现*模式*。但是我们如何确保我们真正发现了*普遍*的模式，而不仅仅是记住了数据？大多数情况下，只有当我们的模型发现这样的模式时，我们的预测才有用。我们不想预测昨天的股票价格，而是明天的。我们不需要识别已经诊断过的疾病对于之前见过的患者，而是要识别之前未诊断的疾病对于之前未见过的患者。这个问题——如何发现可以*泛化*的模式——是机器学习的基本问题，甚至可以说是所有统计学的基本问题。我们可以把这个问题看作是一个更大的问题的一个方面，这个问题涵盖了所有的科学：什么时候我们有理由从特定的观察中跳到更一般的陈述上？

在现实生活中，我们必须使用有限的数据集来拟合我们的模型。这些数据的典型规模在不同领域差异很大。对于许多重要的医学问题，我们只能访问几千个数据点。研究罕见疾病时，我们能幸运地获得几百个数据点就已经不错了。相反，最大的公共数据集，如带有标签的照片，例如ImageNet :cite:`Deng.Dong.Socher.ea.2009`，包含数百万张图片。而一些未标记的图像集合，如Flickr YFC100M数据集，甚至更大，包含超过1亿张图片 :cite:`thomee2016yfcc100m`。然而，即使在这种极端规模下，可用的数据点数量与所有可能的百万像素分辨率图像的空间相比仍然是微不足道的。每当我们在处理有限样本时，必须牢记风险，即我们可能会过度拟合训练数据，结果却发现未能发现可泛化的模式。

比对基础分布更接近训练数据的现象称为*过拟合*，对抗过拟合的技术通常称为*正则化*方法。虽然这不能替代对统计学习理论的适当介绍（见 :citet:`Vapnik98,boucheron2005theory`），但我们将会给你足够的直觉来开始。我们将在本书的许多章节中重新讨论泛化问题，探索关于各种模型背后泛化原理的已知知识，以及在实际任务中发现（经验上）改善泛化的启发式技术。



## 训练误差与泛化误差

在标准的监督学习设置中，我们假设训练数据和测试数据是从*相同*分布中*独立*抽取的。这通常被称为IID假设。尽管这个假设很强，但值得注意的是，如果没有这样的假设，我们将束手无策。为什么我们应该相信从分布$P(X,Y)$中抽样的训练数据应该告诉我们如何对由*不同分布*$Q(X,Y)$生成的测试数据进行预测？做出这样的跳跃需要关于$P$和$Q$之间关系的强烈假设。稍后我们将讨论允许分布偏移的一些假设，但首先我们需要理解IID情况，其中$P(\cdot) = Q(\cdot)$。

首先，我们需要区分*训练误差*$R_\textrm{emp}$，这是在训练数据集上计算的*统计量*，和*泛化误差*$R$，这是相对于基础分布的*期望值*。你可以把泛化误差想象成如果你将你的模型应用到从同一基础数据分布中无限抽取的额外数据示例流上会看到的结果。形式上，训练误差表示为一个*总和*（使用与 :numref:`sec_linear_regression`相同的符号）：

$$R_\textrm{emp}[\mathbf{X}, \mathbf{y}, f] = \frac{1}{n} \sum_{i=1}^n l(\mathbf{x}^{(i)}, y^{(i)}, f(\mathbf{x}^{(i)})),$$


而泛化误差表示为一个积分：

$$R[p, f] = E_{(\mathbf{x}, y) \sim P} [l(\mathbf{x}, y, f(\mathbf{x}))] =
\int \int l(\mathbf{x}, y, f(\mathbf{x})) p(\mathbf{x}, y) \;d\mathbf{x} dy.$$

有问题的是，我们永远无法精确计算泛化误差$R$。没有人会告诉我们密度函数$p(\mathbf{x}, y)$的确切形式。此外，我们不能抽取无限的数据点流。因此，在实践中，我们必须通过将模型应用于一个独立的测试集来*估计*泛化误差，该测试集由随机选择的示例$\mathbf{X}'$和标签$\mathbf{y}'$组成，这些示例和标签被排除在我们的训练集之外。这包括应用与计算经验训练误差相同的公式，但应用于测试集$\mathbf{X}', \mathbf{y}'$。


关键在于，当我们评估分类器在测试集上的表现时，我们是在处理一个*固定的*分类器（它不依赖于测试集的样本），因此估计其误差仅仅是均值估计的问题。然而，对于训练集来说，情况并非如此。请注意，我们最终得到的模型明确依赖于训练集的选择，因此训练误差通常是对基础总体的真实误差（以及泛化误差）的有偏估计。泛化的核心问题是，我们何时应预期我们的训练误差接近总体误差（从而接近泛化误差）。

### 模型复杂度

在经典理论中，当我们有简单的模型和丰富的数据时，训练误差和泛化误差往往很接近。然而，当我们使用更复杂的模型和/或较少的示例时，我们预计训练误差会下降，但泛化差距会扩大。这并不令人惊讶。想象一个非常表达能力强的模型类，对于任意$n$个示例的数据集，我们都可以找到一组参数，完美地拟合任意标签，即使是随机分配的。在这种情况下，即使我们完美地拟合了训练数据，我们又怎么能得出关于泛化误差的结论呢？就我们所知，我们的泛化误差可能并不比随机猜测好多少。

一般来说，如果没有对我们模型类的任何限制，仅根据拟合训练数据，我们不能得出我们的模型发现了任何可泛化的模式 :cite:`vapnik1994measuring`。另一方面，如果我们的模型类没有能力拟合任意标签，那么它一定发现了某种模式。关于模型复杂度的学习理论想法从卡尔·波普尔的思想中得到了一些灵感，卡尔·波普尔是一位有影响力的科学哲学家，他形式化了可证伪性的标准。根据波普尔的说法，一种可以解释任何和所有观察结果的理论根本不是科学理论！毕竟，如果它没有排除任何可能性，那它对世界说了什么呢？简而言之，我们想要的是一个*不能*解释我们可能做出的任何观察结果的假设，但碰巧与我们*实际上*做出的观察结果兼容。

现在，究竟什么是适当的模型复杂度的概念是一个复杂的问题。通常，具有更多参数的模型能够拟合更多的任意分配的标签。然而，这并不总是正确的。例如，核方法在具有无限数量参数的空间中运行，但它们的复杂度通过其他手段控制 :cite:`Scholkopf.Smola.2002`。一种经常证明有用的复杂度概念是参数可以取值的范围。在这里，参数被允许取任意值的模型会更复杂。我们将在下一节中再次讨论这个想法，当我们引入*权重衰减*时，这是你的第一个实用正则化技术。值得注意的是，比较不同模型类成员之间的复杂度可能是困难的（比如说决策树与神经网络）。

此时，我们必须强调另一个重要观点，我们将在引入深度神经网络时再讨论。当一个模型能够拟合任意标签时，低训练误差并不必然意味着低泛化误差。*然而，它也不必然意味着高泛化误差！* 我们可以说得肯定的是，仅凭低训练误差不足以证明低泛化误差。深度神经网络恰好就是这样的模型：虽然它们在实践中泛化得很好，但它们过于强大，以至于我们不能单凭训练误差就得出很多结论。在这些情况下，我们必须更加依赖保留数据来事后认证泛化。在验证集上的误差称为*验证误差*。

## 欠拟合还是过拟合？

当我们比较训练误差和验证误差时，需要注意两种常见的情况。首先，我们要警惕训练误差和验证误差都很高但两者之间差距很小的情况。如果模型无法降低训练误差，这可能意味着我们的模型太简单（即不够表达能力）以捕捉我们试图建模的模式。此外，由于我们的训练误差和泛化误差之间的*泛化差距*（$R_\textrm{emp} - R$）很小，我们有理由相信我们可以采用更复杂的模型。这种现象称为*欠拟合*。

另一方面，正如上面所讨论的，我们要警惕训练误差显著低于验证误差的情况，这表明严重的*过拟合*。请注意，过拟合并不总是坏事。特别是在深度学习中，最好的预测模型通常在训练数据上的表现远远优于保留数据。最终，我们通常关心的是降低泛化误差，并且只在泛化差距成为障碍的情况下才关心差距。请注意，如果训练误差为零，则泛化差距正好等于泛化误差，我们只能通过减少差距来取得进展。

### 多项式曲线拟合
:label:`subsec_polynomial-curve-fitting`

为了说明关于过拟合和模型复杂度的一些经典直觉，请考虑以下情况：给定由单个特征$x$和相应的实值标签$y$组成的训练数据，我们尝试找到$d$次多项式

$$\hat{y}= \sum_{i=0}^d x^i w_i$$

来估计标签$y$。这只是线性回归问题，其中特征由$x$的幂给出，模型的权重由$w_i$给出，偏差由$w_0$给出，因为对于所有$x$，$x^0 = 1$。由于这是一个线性回归问题，我们可以使用平方误差作为损失函数。

更高阶的多项式函数比更低阶的多项式函数更复杂，因为更高阶的多项式有更多的参数，模型函数的选择范围更广。固定训练数据集，更高阶的多项式函数相对于低阶多项式应该始终实现更低（最坏情况下相等）的训练误差。事实上，每当每个数据示例都有不同的$x$值时，一个次数等于数据示例数量的多项式函数可以完美地拟合训练集。我们在 :numref:`fig_capacity_vs_error` 中比较了多项式次数（模型复杂度）与欠拟合和过拟合之间的关系。

![模型复杂度对欠拟合和过拟合的影响。](../img/capacity-vs-error.svg)
:label:`fig_capacity_vs_error`


### 数据集大小

正如上述边界已经表明的那样，另一个需要牢记的重要因素是数据集大小。固定模型，我们在训练数据集中拥有的样本越少，遇到过拟合的可能性越大（并且程度越严重）。随着训练数据量的增加，泛化误差通常会减少。此外，一般来说，更多的数据永远不会有害。对于固定的任务和数据分布，模型复杂度不应比数据量增长得更快。有了更多的数据，我们可能会尝试拟合更复杂的模型。缺乏足够的数据，简单的模型可能更难超越。对于许多任务，深度学习只有在数千个训练示例可用时才能超越线性模型。部分原因是当前深度学习的成功很大程度上归功于来自互联网公司、廉价存储、连接设备和经济广泛数字化的大规模数据集的丰富。

## 模型选择
:label:`subsec_generalization-model-selection`

通常，我们在评估多个在各个方面有所不同的模型（不同的架构、训练目标、选定的特征、数据预处理、学习率等）之后才会选择最终模型。在多个模型之间进行选择恰当地称为*模型选择*。

原则上，在选择了所有超参数之前，我们不应该触碰测试集。如果我们使用测试数据进行模型选择，存在过拟合测试数据的风险。然后我们会陷入严重的困境。如果我们过拟合了训练数据，总有在测试数据上的评估来让我们保持诚实。但如果过拟合了测试数据，我们怎么知道？请参阅 :citet:`ong2005learning` 了解一个例子，说明即使对于可以严格控制复杂度的模型，这也可能导致荒谬的结果。

因此，我们绝不应依赖测试数据进行模型选择。而且，我们也不能仅仅依赖训练数据进行模型选择，因为我们无法在用于训练模型的数据上估计泛化误差。

在实际应用中，情况变得更加模糊。虽然理想情况下我们只会使用一次测试数据，来评估最佳模型或比较少数几个模型，但在现实世界中，测试数据很少在一次使用后就被丢弃。我们很少能在每轮实验中负担得起一个新的测试集。事实上，几十年来回收基准数据可以对算法的发展产生重大影响，例如[图像分类](https://paperswithcode.com/sota/image-classification-on-imagenet)和[光学字符识别](https://paperswithcode.com/sota/image-classification-on-mnist)。

解决*在测试集上训练*问题的常见做法是将数据分为三部分，除了训练集和测试集外，还包括*验证集*。结果是一个模糊的业务，验证数据和测试数据之间的界限令人担忧地模糊不清。除非另有明确说明，本书中的实验实际上是使用应该正确称为训练数据和验证数据的内容，没有真正的测试集。因此，书中每个实验报告的准确性实际上是验证准确性，而不是真正的测试集准确性。

### 交叉验证

当训练数据稀缺时，我们可能连保留足够的数据构成适当的验证集都负担不起。解决这个问题的一种流行方法是采用$K$*折交叉验证*。这里，原始训练数据被分成$K$个不重叠的子集。然后执行$K$次模型训练和验证，每次在$K-1$个子集上训练，并在一个不同的子集（该轮未用于训练的子集）上验证。最后，通过对$K$次实验的结果求平均来估计训练和验证误差。



## 总结

本节探讨了机器学习中泛化的一些基础。当涉及到更深层次的模型时，其中一些想法变得复杂且反直觉；在这里，模型能够严重过拟合数据，相关的复杂度概念可能是隐含的且反直觉的（例如，具有更多参数的更大架构泛化得更好）。我们留给你一些经验法则：

1. 使用验证集（或$K$*折交叉验证*）进行模型选择；
1. 更复杂的模型通常需要更多的数据；
1. 相关的复杂度概念既包括参数的数量，也包括参数允许取值的范围；
1. 在其他条件相同的情况下，更多的数据几乎总是会导致更好的泛化；
1. 所有关于泛化的讨论都是基于IID假设的前提下的。如果我们放松这个假设，允许分布在训练和测试期间发生变化，那么在没有进一步（也许是较温和的）假设的情况下，我们就无法说任何关于泛化的事情。


## 练习

1. 你什么时候可以精确解决多项式回归问题？
1. 给出至少五个相关随机变量使得将问题视为IID数据不合适的情况。
1. 你是否可以期待看到零训练误差？在什么情况下你会看到零泛化误差？
1. 为什么$K$-折交叉验证计算成本非常高？
1. 为什么$K$-折交叉验证误差估计是有偏的？
1. VC维度定义为一类函数中可以将任意标签$\{\pm 1\}$分类的最大点数。为什么这不是衡量函数类复杂度的好方法？提示：考虑函数的幅度。
1. 你的经理给了你一个难以处理的数据集，你的当前算法在这个数据集上的表现不佳。你怎么向他证明你需要更多的数据？提示：你不能增加数据，但可以减少数据。

[讨论](https://discuss.d2l.ai/t/97)